<a href="https://colab.research.google.com/github/netrapathak/dataria-capstone/blob/master/NN_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from cleanco import cleanco
import re
import unicodedata
!pip install ftfy
from ftfy import fix_text
import sys
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

In [0]:
##Import dataframes here. 
#Pitchbook data
pitch = pd.read_csv('Pitchbook.csv', dtype='unicode')
#USPTO data
uspto = pd.read_csv('uspto.csv', dtype='unicode')

In [0]:
def ngrams(string, n=3):
    string = fix_text(string) 
    string = string.encode("ascii", errors="ignore").decode() 
    string = string.lower() 
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) 
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() 
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string +' ' 
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [0]:
#This is the record linkage approach. Use this approach. 
#Company name column from pitchbook.
org_name_clean = pitch['Company Legal Name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(org_name_clean.astype('U'))
print('Vecorizing completed...')
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
#Company name column from uspto.
org_column = uspto['organization'] #column to match against in the messy data

unique_org = set(org_column.astype('U')) 

def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)
unique_org = list(unique_org) 

In [0]:
matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), pitch.values[j][0][1],unique_org[i]]
    matches.append(temp)
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Original name'])
print('Done')

In [0]:
match = matches.sort_values(by=['Match confidence (lower is better)'])
match_1 = match[match['Match confidence (lower is better)'] < 0.79]

Use match_1['Matched Name'] to join with pitchbook dataset.

